In [1]:
#I've made a fasta file of the barcodes for each amplicon ITS and LSU, this is what is in the folder
!ls

BM.ITS.barcodes.fasta    BM_ITS_barcodes.txt      BeetleMania_04122017.bam
BM.LSU.barcodes.fasta    BM_LSU_barcodes.txt      Untitled.ipynb


In [3]:
#I'll then demultiplex, first lets check primers that are hardcoded into AMPtk
#you could also specify primer sequences directly, i.e. -f GTGARTCATCGAATCTTTG
!amptk primers

----------------------------------
Primers hard-coded into AMPtk:
----------------------------------
16S_V3       CCTACGGGNGGCWGCAG
16S_V4       GACTACHVGGGTATCTAATCC
COI-F        GGTCAACAAATCATAAAGATATTGG
COI-R        GGWACTAATCAATTTCCAAATCC
ITS1         TCCGTAGGTGAACCTGCGG
ITS1-F       CTTGGTCATTTAGAGGAAGTAA
ITS2         GCTGCGTTCTTCATCGATGC
ITS3         GCATCGATGAAGAACGCAGC
ITS3_KYO2    GATGAAGAACGYAGYRAA
ITS4         TCCTCCGCTTATTGATATGC
ITS4-B       CAGGAGACTTGTACACGGTCCAG
JH-LS-369rc  CTTCCCTTTCAACAATTTCAC
LR0R         ACCCGCTGAACTTAAGC
LR2R         AAGAACTTTGAAAAGAG
fITS7        GTGARTCATCGAATCTTTG
----------------------------------


In [5]:
#We've used fITS7 and ITS4 for generating the ITS2 amplicons, so will demulitplex like this
#which will demux barcodes, remove primers, and trim/pad amplicons to 250 bp
#shortest known ITS2 sequences are around 150 bp, so we will drop all reads less than 125 to get rid of 
#primer dimers and other likely non-biological sequences
!amptk ion -i BeetleMania_04122017.bam -o its -f fITS7 -r ITS4 --barcode_fasta BM.ITS.barcodes.fasta --min_len 125 

-------------------------------------------------------
[07:52:38 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[07:52:38 AM]: amptk v.0.9.0, USEARCH v9.2.64, VSEARCH v2.4.2
[07:52:38 AM]: Converting Ion Torrent BAM file to FASTQ using BedTools
[07:54:54 AM]: Foward primer: AGTGARTCATCGAATCTTTG,  Rev comp'd rev primer: GCATATCAATAAGCGGAGGA
[07:54:54 AM]: Loading FASTQ Records
[07:55:01 AM]: 6,122,373 reads (4.7 GB)
-------------------------------------------------------
[08:41:36 AM]: Concatenating Demuxed Files
[08:42:06 AM]: 6,122,373 total reads
[08:42:06 AM]: 2,923,103 valid Barcode
[08:42:06 AM]: 2,618,782 Fwd Primer found, 2,202,811 Rev Primer found
[08:42:06 AM]: 174,159 discarded too short (< 125 bp)
[08:42:06 AM]: 2,444,623 valid output reads
[08:42:13 AM]: Found 46 barcoded samples
                Sample:  Count
                 BC.73:  96325
                 BC.79:  82931
                 BC.75:  82834
               SynMock:  81122
                 BC.87:  8

##### Looks like above code ran well and we have a range of ~15,000 to 95,000 sequences from each sample.  Despite trying hard to get this to be equal, it never will be.  Usually we try to have at least 10,000 reads per sample.  You can also see that the script has created a QIIME-like mapping file, which we can edit to add our meta data to.

In [10]:
#first lets cluster the data using UPARSE to generate OTUs
!amptk cluster -i its.demux.fq -o its

-------------------------------------------------------
[09:44:22 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[09:44:22 AM]: amptk v.0.9.0, USEARCH v9.2.64, VSEARCH v2.4.2
[09:44:22 AM]: Loading FASTQ Records
[09:44:24 AM]: 2,444,623 reads (1.2 GB)
[09:44:24 AM]: Quality Filtering, expected errors < 1.0
[09:45:14 AM]: 1,174,205 reads passed
[09:45:14 AM]: De-replication (remove duplicate reads)
[09:45:17 AM]: 86,475 reads passed
[09:45:18 AM]: Clustering OTUs (UPARSE)
[09:45:20 AM]: 397 OTUs
[09:45:20 AM]: Cleaning up padding from OTUs
[09:45:20 AM]: Mapping Reads to OTUs and Building OTU table
[09:49:31 AM]: 2,036,787 reads mapped to OTUs (83%)
-------------------------------------------------------
OTU Clustering Script has Finished Successfully
-------------------------------------------------------
Clustered OTUs: /Users/jon/projects/beetlemania/its.cluster.otus.fa
OTU Table: /Users/jon/projects/beetlemania/its.otu_table.txt
---------------------------------------

In [11]:
#since we have a mock in this run, specifically synthetic mock, we can filter dataset
!amptk filter -i its.otu_table.txt -f its.cluster.otus.fa -b SynMock -m synmock

-------------------------------------------------------
[09:50:00 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[09:50:00 AM]: amptk v.0.9.0, USEARCH v9.2.64, VSEARCH v2.4.2
[09:50:00 AM]: Loading OTU table: its.otu_table.txt
[09:50:00 AM]: OTU table contains 397 OTUs
[09:50:00 AM]: Mapping OTUs to Mock Community (USEARCH)
[09:50:00 AM]: Sorting OTU table naturally
[09:50:00 AM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
[09:50:00 AM]: Normalizing OTU table to number of reads per sample
[09:50:00 AM]: Index bleed, mock into samples: 14.225539%.  Index bleed, samples into mock: 0.016000%.
[09:50:00 AM]: Will use value of 14.300000% for index-bleed OTU filtering.
[09:50:00 AM]: Index bleed into samples is abnormally high (14.300000%), if you have biological mock you should use `--calculate in`
[09:50:00 AM]: Filtering OTU table down to 389 OTUs
[09:50:00 AM]: Filtering valid OTUs
--------------------------------------------

In [12]:
#this result is indicative of contamination, not index-bleed. Index-bleed on Ion happens ~ 0.2%.
#While we used the synthetic mock, we will modify for right now
!amptk filter -i its.otu_table.txt -f its.cluster.otus.fa -b SynMock -m synmock --calculate in

-------------------------------------------------------
[09:55:57 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[09:55:57 AM]: amptk v.0.9.0, USEARCH v9.2.64, VSEARCH v2.4.2
[09:55:57 AM]: Loading OTU table: its.otu_table.txt
[09:55:57 AM]: OTU table contains 397 OTUs
[09:55:57 AM]: Mapping OTUs to Mock Community (USEARCH)
[09:55:57 AM]: Sorting OTU table naturally
[09:55:57 AM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
[09:55:57 AM]: Normalizing OTU table to number of reads per sample
[09:55:57 AM]: Index bleed, samples into mock: 0.016000%.
[09:55:57 AM]: Will use value of 0.100000% for index-bleed OTU filtering.
[09:55:58 AM]: Filtering OTU table down to 391 OTUs
[09:55:58 AM]: Filtering valid OTUs
-------------------------------------------------------
OTU Table filtering finished
-------------------------------------------------------
OTU Table Stats:      its.stats.txt
Sorted OTU table:     its.sorted.txt
Normalize

In [29]:
#because of the contamination from the synthetic mock into a few of the other samples, one could either drop those 
#samples or we could just drop those OTUs, I'll just quickly drop those OTUs for the time being
!grep -v '^mock' its.final.txt > its.final.clean.txt

In [14]:
#how lets have a look at the mapping file that the script made for us
!head its.mapping_file.txt

#SampleID	BarcodeSequence	LinkerPrimerSequence	ReversePrimer	phinchID	Treatment
BC.49	TCCTAACATAAC	CCATCTCATCCCTGCGTGTCTCCGACTCAGTCCTAACATAACAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.49	no_data
BC.50	CGGACAATGGC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCGGACAATGGCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.50	no_data
BC.51	TTGAGCCTATTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGTTGAGCCTATTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.51	no_data
BC.52	CCGCATGGAAC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCCGCATGGAACAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.52	no_data
BC.53	CTGGCAATCCTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCTGGCAATCCTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.53	no_data
BC.54	CCGGAGAATCGC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCCGGAGAATCGCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.54	no_data
BC.55	TCCACCTCCTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGTCCACCTCCTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.55	no_data
BC.56	CAGCATTAATTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCAGCATTAATTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	BC.5

#### Open this file in excel or text editor to add your meta data, can be any number of columns

In [16]:
#added meta data, lets see what that looks like, i but the time T0 and T1 into Treatment section
!head its.meta.mapping_file.txt

#SampleID	BarcodeSequence	LinkerPrimerSequence	ReversePrimer	phinchID	Treatment	cage	section
BC.49	TCCTAACATAAC	CCATCTCATCCCTGCGTGTCTCCGACTCAGTCCTAACATAACAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	T0_1_T	T0	1	T
BC.50	CGGACAATGGC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCGGACAATGGCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	T0_1_M	T0	1	M
BC.51	TTGAGCCTATTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGTTGAGCCTATTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	T0_1_B	T0	1	B
BC.52	CCGCATGGAAC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCCGCATGGAACAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	T0_2_T	T0	2	T
BC.53	CTGGCAATCCTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCTGGCAATCCTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	T0_2_M	T0	2	M
BC.54	CCGGAGAATCGC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCCGGAGAATCGCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	T0_2_B	T0	2	B
BC.55	TCCACCTCCTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGTCCACCTCCTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTATTGATATGC	T0_3_T	T0	3	T
BC.56	CAGCATTAATTC	CCATCTCATCCCTGCGTGTCTCCGACTCAGCAGCATTAATTCAGTGARTCATCGAATCTTTG	TCCTCCGCTTAT

In [30]:
#now that we have filtered OTU table, filtered OTUs, and a meta mapping file, we will run taxonomy
!amptk taxonomy -f its.filtered.otus.fa -i its.final.clean.txt -m its.meta.mapping_file.txt -d ITS2

-------------------------------------------------------
[03:11:10 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[03:11:10 PM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[03:11:10 PM]: Loading FASTA Records
[03:11:10 PM]: 391 OTUs
[03:11:10 PM]: Global alignment OTUs with usearch_global (USEARCH)
[03:11:15 PM]: Classifying OTUs with UTAX (USEARCH)
[03:11:15 PM]: Classifying OTUs with SINTAX (USEARCH)
[03:12:39 PM]: Appending taxonomy to OTU table and OTUs
[03:12:39 PM]: Generating phylogenetic tree
[03:12:40 PM]: Taxonomy finished: its.taxonomy.txt
[03:12:40 PM]: Classic OTU table with taxonomy: its.otu_table.taxonomy.txt
[03:12:41 PM]: BIOM OTU table created: its.biom
[03:12:41 PM]: OTUs with taxonomy: its.otus.taxonomy.fa
[03:12:41 PM]: OTU phylogeny: its.tree.phy
-------------------------------------------------------


In [19]:
#Okay now we have taxonomy assigned to our ITS data, we can see that by looking at first few records
!head its.otus.taxonomy.fa

>OTU1 EF445342;k:Fungi,p:Ascomycota,c:Dothideomycetes,o:Botryosphaeriales,f:Botryosphaeriaceae,g:Sphaeropsis,s:Sphaeropsis sapinea
AACGCACATTGCGCCCCTTGGCATTCCGAGGGGCATGCCTGTTCGAGCGTCATTACAACC
CTCAAGCTCTGCTTGGTATTGGGCGCCGTCCTCTCTGCGGACGCGCCTTAAAGACCTCGG
CGGTGGCTGTTCAGCCCTCAAGCGTAGTAGAATACACCTCGCTTTGGAGCGGTTGGCGTC
GCCCGCCGGACGAACCTTCTGAACTTTTCTCAAGGTTGACCTCGGATCAGGTAGGGATAC
CCGCTGAACT
>OTU2 FJ627241;k:Fungi,p:Basidiomycota,c:Agaricomycetes,o:Russulales,f:Bondarzewiaceae,g:Heterobasidion
AACGCACCTTGCGCCCTTTGGTATTCCGAAGGGCACGCCTGTTTGAGTGTCGTGAAATTC
TCAACCCCGTGCTTTTCTTGTGAAAGCGCGTGGGCTTGGACTTGGAGGTTTTGCTGGTCC
TCGCGGATCGGCTCCTCTCAAATGCATTAGCGAGACCCTTGTGGTGCCGTCCCCGGTGTG


### Okay, now onto LSU data set

In [6]:
#since I'm less sure of the size for LSU, I'll run first batch of only keeping full length reads
#I will re-use the fastq file from ITS run, note it was just converted from BAM to FASTQ, so save a few minutes doing
#the conversion again.  LSU should have less length variation than ITS, perhaps full length will work well
#if not, we will run again at a shorter length
!amptk ion -i its.fastq -o lsu -f LR0R -r JH-LS-369rc --barcode_fasta BM.LSU.barcodes.fasta --min_len 200 --full_length

-------------------------------------------------------
[08:43:00 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[08:43:00 AM]: amptk v.0.9.0, USEARCH v9.2.64, VSEARCH v2.4.2
[08:43:00 AM]: Foward primer: AACCCGCTGAACTTAAGC,  Rev comp'd rev primer: GTGAAATTGTTGAAAGGGAAG
[08:43:00 AM]: Loading FASTQ Records
[08:43:07 AM]: 6,122,373 reads (4.7 GB)
-------------------------------------------------------
[09:32:39 AM]: Concatenating Demuxed Files
[09:33:03 AM]: 6,122,373 total reads
[09:33:03 AM]: 2,360,851 valid Barcode
[09:33:03 AM]: 2,269,140 Fwd Primer found, 1,577,089 Rev Primer found
[09:33:03 AM]: 4,195 discarded too short (< 200 bp)
[09:33:03 AM]: 1,572,894 valid output reads
[09:33:09 AM]: Found 49 barcoded samples
                Sample:  Count
                 BC.34:  42490
                 BC.36:  41083
                 BC.46:  39659
                 BC.30:  39500
                 BC.48:  38524
                  BC.9:  37994
                 BC.23:  37223
       

In [8]:
#lets see what the read length distribution is in this LSU sample
!amptk show -i lsu.demux.fq

----------------------------------
Found 49 barcoded samples
              Sample:  Count
               BC.34:  42490
               BC.36:  41083
               BC.46:  39659
               BC.30:  39500
               BC.48:  38524
                BC.9:  37994
               BC.23:  37223
               BC.43:  37198
               BC.66:  37191
                BC.7:  36869
               BC.37:  36599
               BC.11:  36453
               BC.10:  36172
               BC.19:  35983
               BC.14:  35159
               BC.39:  35159
               BC.18:  35005
               BC.24:  34758
               BC.20:  34322
               BC.40:  33948
               BC.28:  33807
                BC.3:  33784
               BC.45:  33711
               BC.27:  33553
               BC.41:  33529
               BC.22:  33230
               BC.47:  33193
                BC.8:  33014
               BC.16:  33008
               BC.32:  32964
               BC.26:  32742
           

In [9]:
#we can see from here that the average length is ~ 396 bp, and we have some suspiciously long sequences as well. 
#remember that we used 400 bp sequencing kit, which can get out to about 450 bp, so for right now I will leave those
#those sample in.  I would also drop BC.13 here as the number of reads is too low, so lets do that now like this
!amptk remove -i lsu.demux.fq -l BC.13 -o lsu.clean.fq

Removed 1 samples
Kept 1572383 reads out of 1572894 total reads


In [38]:
#because there is less variation in LSU region, I'll try to run DADA2 denoising which can differentiate between single
#base pair differences. because the read length is longer, I'm increasing the maxEE quality filter value
!amptk dada2 -i lsu.clean.fq -o lsu -e 2.0 --pool

-------------------------------------------------------
[03:34:29 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[03:34:29 PM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[03:34:29 PM]: R v3.3.3; DADA2 v1.3.3
[03:34:29 PM]: Loading FASTQ Records
[03:34:31 PM]: 1,572,383 reads (1.1 GB)
[03:34:46 PM]: Quality Filtering, expected errors < 2.0
[03:34:46 PM]: 11,463 reads passed
[03:34:46 PM]: Splitting FASTQ file by Sample into individual files
[03:34:47 PM]: Running DADA2 pipeline
[03:36:09 PM]: 90 total inferred sequences (iSeqs)
[03:36:09 PM]: 1 denovo chimeras removed
[03:36:09 PM]: 89 valid iSeqs
[03:36:09 PM]: Mapping reads to DADA2 iSeqs
[03:40:48 PM]: 1,420,504 reads mapped to iSeqs (90%)
[03:40:48 PM]: Clustering iSeqs at 97% to generate biological OTUs
[03:40:48 PM]: 76 OTUs generated
[03:40:48 PM]: Mapping reads to OTUs
[03:45:12 PM]: 1,404,335 reads mapped to OTUs (89%)
-------------------------------------------------------
DADA2 Script has Finished Successf

In [40]:
#we see that probably too few reads have passed the quality filter.  However, you can also see that 90% of the reads
#mapped to those iSeqs, so this might be okay.  But because of the long length of the reads using full length 
#we are likely getting a lot of low quality basepairs near the 3' end of the reads. Also there were some really long reads
#So I'm going to go back and re-do the demuxing and do a trim/pad
#length of 350 bp, so we can get rid of those longer reads which might be problematic but will also keep reads where 
# we did not find the reverse primer, trimming to 350 seems reasonable.
!amptk ion -i its.fastq -o lsu2 -f LR0R -r JH-LS-369rc --barcode_fasta BM.LSU.barcodes.fasta --min_len 250 -l 350

-------------------------------------------------------
[03:57:43 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[03:57:43 PM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[03:57:43 PM]: Foward primer: AACCCGCTGAACTTAAGC,  Rev comp'd rev primer: GTGAAATTGTTGAAAGGGAAG
[03:57:43 PM]: Loading FASTQ Records
[03:57:48 PM]: 6,122,373 reads (4.7 GB)
-------------------------------------------------------
[04:46:24 PM]: Concatenating Demuxed Files
[04:46:56 PM]: 6,122,373 total reads
[04:46:56 PM]: 2,360,851 valid Barcode
[04:46:56 PM]: 2,269,140 Fwd Primer found, 1,577,089 Rev Primer found
[04:46:56 PM]: 49,907 discarded too short (< 250 bp)
[04:46:56 PM]: 2,219,233 valid output reads
[04:47:04 PM]: Found 49 barcoded samples
                Sample:  Count
                 BC.30:  62570
                 BC.46:  54578
                 BC.19:  54354
                 BC.45:  54222
                  BC.5:  53679
                 BC.48:  53023
                 BC.40:  53015
      

In [41]:
#clean up, remove BC.13
!amptk remove -i lsu2.demux.fq -l BC.13 -o lsu2.clean.fq

Removed 1 samples
Kept 2218606 reads out of 2219233 total reads


In [42]:
#run dada2 again
!amptk dada2 -i lsu2.clean.fq -o lsu2 -e 2.0 --pool

-------------------------------------------------------
[04:51:15 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[04:51:15 PM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[04:51:16 PM]: R v3.3.3; DADA2 v1.3.3
[04:51:16 PM]: Loading FASTQ Records
[04:51:17 PM]: 2,218,606 reads (1.5 GB)
[04:51:36 PM]: Quality Filtering, expected errors < 2.0
[04:51:37 PM]: 15,809 reads passed
[04:51:37 PM]: Splitting FASTQ file by Sample into individual files
[04:51:38 PM]: Running DADA2 pipeline
[04:53:37 PM]: 118 total inferred sequences (iSeqs)
[04:53:37 PM]: 1 denovo chimeras removed
[04:53:37 PM]: 117 valid iSeqs
[04:53:37 PM]: Mapping reads to DADA2 iSeqs
[05:26:41 PM]: 1,818,479 reads mapped to iSeqs (82%)
[05:26:41 PM]: Clustering iSeqs at 97% to generate biological OTUs
[05:26:41 PM]: 94 OTUs generated
[05:26:41 PM]: Mapping reads to OTUs
[05:33:18 PM]: 1,789,716 reads mapped to OTUs (81%)
-------------------------------------------------------
DADA2 Script has Finished Succes

In [43]:
#we don't have mock community here, but we will still run a filter at 0.1% (same as calculated from ITS)
#here we will use the DADA2 inferred sequences first
!amptk filter -i lsu2.otu_table.txt -f lsu2.iSeqs.fa -p 0.001

-------------------------------------------------------
[05:40:35 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[05:40:35 PM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[05:40:35 PM]: Loading OTU table: lsu2.otu_table.txt
[05:40:35 PM]: OTU table contains 117 OTUs
[05:40:35 PM]: Sorting OTU table naturally
[05:40:35 PM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
[05:40:35 PM]: Normalizing OTU table to number of reads per sample
[05:40:35 PM]: Overwriting auto detect index-bleed, setting to 0.100000%
[05:40:35 PM]: Filtering OTU table down to 117 OTUs
[05:40:35 PM]: Filtering valid OTUs
-------------------------------------------------------
OTU Table filtering finished
-------------------------------------------------------
OTU Table Stats:      lsu2.stats.txt
Sorted OTU table:     lsu2.sorted.txt
Normalized/filter:    lsu2.normalized.txt
Final Binary table:   lsu2.final.binary.txt
Final OTU table:      lsu2.final.tx

In [44]:
#I did same thing for LSU mapping file as for ITS, added Meta Data
!amptk taxonomy -f lsu2.filtered.otus.fa -i lsu2.final.txt -m lsu.meta.mapping_file.txt -d LSU

-------------------------------------------------------
[05:44:43 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[05:44:43 PM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[05:44:43 PM]: Loading FASTA Records
[05:44:43 PM]: 117 OTUs
[05:44:43 PM]: Global alignment OTUs with usearch_global (USEARCH)
[05:44:44 PM]: Classifying OTUs with UTAX (USEARCH)
[05:44:44 PM]: Classifying OTUs with SINTAX (USEARCH)
[05:44:57 PM]: Appending taxonomy to OTU table and OTUs
[05:44:57 PM]: Generating phylogenetic tree
[05:44:57 PM]: Taxonomy finished: lsu2.taxonomy.txt
[05:44:57 PM]: Classic OTU table with taxonomy: lsu2.otu_table.taxonomy.txt
[05:44:58 PM]: BIOM OTU table created: lsu2.biom
[05:44:58 PM]: OTUs with taxonomy: lsu2.otus.taxonomy.fa
[05:44:58 PM]: OTU phylogeny: lsu2.tree.phy
-------------------------------------------------------


In [2]:
#try to run dada2 again, had this fastq_qmin=2 setting in Q filtering, wondering if that dropped unnecessary reads.
#yes it did, so these results should be more robust, it also took 7 hours to run....
!amptk dada2 -i lsu2.clean.fq -o lsu3 -e 2.0 --pool 

-------------------------------------------------------
[07:24:13 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[07:24:13 PM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[07:24:13 PM]: R v3.3.3; DADA2 v1.3.3
[07:24:13 PM]: Loading FASTQ Records
[07:24:15 PM]: 2,218,606 reads (1.5 GB)
[07:24:36 PM]: Quality Filtering, expected errors < 2.0
[07:25:07 PM]: 1,207,703 reads passed
[07:25:07 PM]: Splitting FASTQ file by Sample into individual files
[07:26:27 PM]: Running DADA2 pipeline
[03:57:07 AM]: 617 total inferred sequences (iSeqs)
[03:57:07 AM]: 209 denovo chimeras removed
[03:57:07 AM]: 408 valid iSeqs
[03:57:07 AM]: Mapping reads to DADA2 iSeqs
[04:03:52 AM]: 1,878,655 reads mapped to iSeqs (85%)
[04:03:52 AM]: Clustering iSeqs at 97% to generate biological OTUs
[04:03:52 AM]: 197 OTUs generated
[04:03:52 AM]: Mapping reads to OTUs
[04:10:17 AM]: 1,843,273 reads mapped to OTUs (83%)
-------------------------------------------------------
DADA2 Script has Finished 

In [3]:
#now filter the iSeqs at 0.1%
!amptk filter -i lsu3.otu_table.txt -f lsu3.iSeqs.fa -p 0.001

-------------------------------------------------------
[08:07:27 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[08:07:27 AM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[08:07:27 AM]: Loading OTU table: lsu3.otu_table.txt
[08:07:27 AM]: OTU table contains 408 OTUs
[08:07:27 AM]: Sorting OTU table naturally
[08:07:27 AM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
[08:07:27 AM]: Normalizing OTU table to number of reads per sample
[08:07:27 AM]: Overwriting auto detect index-bleed, setting to 0.100000%
[08:07:28 AM]: Filtering OTU table down to 408 OTUs
[08:07:28 AM]: Filtering valid OTUs
-------------------------------------------------------
OTU Table filtering finished
-------------------------------------------------------
OTU Table Stats:      lsu3.stats.txt
Sorted OTU table:     lsu3.sorted.txt
Normalized/filter:    lsu3.normalized.txt
Final Binary table:   lsu3.final.binary.txt
Final OTU table:      lsu3.final.tx

In [5]:
!amptk taxonomy -f lsu3.filtered.otus.fa -i lsu3.final.txt -m lsu.meta.mapping_file.txt -d LSU

-------------------------------------------------------
[08:08:04 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[08:08:04 AM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[08:08:04 AM]: Loading FASTA Records
[08:08:04 AM]: 408 OTUs
[08:08:04 AM]: Global alignment OTUs with usearch_global (USEARCH)
[08:08:06 AM]: Classifying OTUs with UTAX (USEARCH)
[08:08:06 AM]: Classifying OTUs with SINTAX (USEARCH)
[08:08:27 AM]: Appending taxonomy to OTU table and OTUs
[08:08:27 AM]: Generating phylogenetic tree
[08:08:28 AM]: Taxonomy finished: lsu3.taxonomy.txt
[08:08:28 AM]: Classic OTU table with taxonomy: lsu3.otu_table.taxonomy.txt
[08:08:29 AM]: BIOM OTU table created: lsu3.biom
[08:08:29 AM]: OTUs with taxonomy: lsu3.otus.taxonomy.fa
[08:08:29 AM]: OTU phylogeny: lsu3.tree.phy
-------------------------------------------------------


In [6]:
#also filter the clustered dada2 output to see what it looks like
!amptk filter -i lsu3.cluster.otu_table.txt -o lsu3.cluster -f lsu3.cluster.otus.fa -p 0.001

-------------------------------------------------------
[08:13:28 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[08:13:28 AM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[08:13:28 AM]: Loading OTU table: lsu3.cluster.otu_table.txt
[08:13:28 AM]: OTU table contains 197 OTUs
[08:13:28 AM]: Sorting OTU table naturally
[08:13:28 AM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
[08:13:28 AM]: Normalizing OTU table to number of reads per sample
[08:13:28 AM]: Overwriting auto detect index-bleed, setting to 0.100000%
[08:13:29 AM]: Filtering OTU table down to 197 OTUs
[08:13:29 AM]: Filtering valid OTUs
-------------------------------------------------------
OTU Table filtering finished
-------------------------------------------------------
OTU Table Stats:      lsu3.cluster.stats.txt
Sorted OTU table:     lsu3.cluster.sorted.txt
Normalized/filter:    lsu3.cluster.normalized.txt
Final Binary table:   lsu3.cluster.final.binary

In [7]:
!amptk taxonomy -f lsu3.cluster.filtered.otus.fa -i lsu3.cluster.final.txt -m lsu.meta.mapping_file.txt -d LSU

-------------------------------------------------------
[08:13:55 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[08:13:55 AM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.2
[08:13:55 AM]: Loading FASTA Records
[08:13:55 AM]: 197 OTUs
[08:13:55 AM]: Global alignment OTUs with usearch_global (USEARCH)
[08:13:56 AM]: Classifying OTUs with UTAX (USEARCH)
[08:13:56 AM]: Classifying OTUs with SINTAX (USEARCH)
[08:14:14 AM]: Appending taxonomy to OTU table and OTUs
[08:14:14 AM]: Generating phylogenetic tree
[08:14:14 AM]: Taxonomy finished: lsu3.cluster.taxonomy.txt
[08:14:14 AM]: Classic OTU table with taxonomy: lsu3.cluster.otu_table.taxonomy.txt
[08:14:15 AM]: BIOM OTU table created: lsu3.cluster.biom
[08:14:15 AM]: OTUs with taxonomy: lsu3.cluster.otus.taxonomy.fa
[08:14:15 AM]: OTU phylogeny: lsu3.cluster.tree.phy
-------------------------------------------------------
